In [2]:
!pip install python-docx


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 5.5 MB/s eta 0:00:00


In [4]:
!pip install PyPDF2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.1 MB/s eta 0:00:00


In [5]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from docx import Document
from PyPDF2 import PdfFileReader
import torch
from transformers import BertTokenizer, BertModel
import numpy as np

In [6]:
# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [7]:
# Load pretrained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [17]:
# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text()
        return text

In [9]:
# Function to extract text from DOCX
def extract_text_from_docx(docx_path):
    doc = Document(docx_path)
    text = []
    for paragraph in doc.paragraphs:
        text.append(paragraph.text)
    return '\n'.join(text)

In [10]:
# Function to extract text from file
def extract_text_from_file(file_path):
    if file_path.endswith('.pdf'):
        return extract_text_from_pdf(file_path)
    elif file_path.endswith('.docx'):
        return extract_text_from_docx(file_path)
    else:
        raise ValueError("Unsupported file format")

In [11]:
# Function to preprocess text
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove non-alphabetic characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenize text
    tokens = word_tokenize(text)
    # Define stop words
    stop_words = set(stopwords.words('english'))
    # Remove stop words
    tokens = [word for word in tokens if word not in stop_words]
    # Lemmatize words
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    # Join tokens back into a string
    clean_text = ' '.join(tokens)
    return clean_text

In [12]:
# Function to calculate similarity using BERT
def calculate_similarity(job_description, resume_text):
    # Preprocess the job description and resume text
    job_description_cleaned = preprocess_text(job_description)
    resume_text_cleaned = preprocess_text(resume_text)

    # Tokenize and encode the texts
    inputs = tokenizer([job_description_cleaned, resume_text_cleaned], return_tensors='pt', padding=True, truncation=True)

    # Get BERT embeddings
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the embeddings for the [CLS] token (first token) which is used for classification tasks
    job_desc_embedding = outputs.last_hidden_state[0][0]
    resume_embedding = outputs.last_hidden_state[1][0]

    # Calculate cosine similarity
    cosine_sim = torch.nn.functional.cosine_similarity(job_desc_embedding, resume_embedding, dim=0)

    return cosine_sim.item()

In [18]:
from PyPDF2 import PdfReader

In [19]:
if __name__ == "__main__":
    job_description = """
    Expertise in at least one programming language Python, R etc. and related data science tools (e.g. Pandas, Scikit-learn)
    Experience with SQL databases
    Proficiency in machine learning frameworks, including TensorFlow, Py-Torch, or Scikit-learn, CI/CD pipelines, and containerization.
    Experience in AI-driven conversational interfaces, crafting chatbots that engage and assist users seamlessly.
    Design and implement conversational interfaces for varied applications.
    Collaborate with UX/UI designers to enhance chatbot user experience.
    Integrate chatbots with various platforms and backend systems.
    Familiarity with cloud computing platforms such as Microsoft Azure, Databricks, AWS, Google Cloud, or snowflake, and their machine learning services
    Excellent communication skills and ability to collaborate effectively with cross-functional teams.
    """

    resume_path = "/content/Vyankatesh_pophale_B.tech_compressed.pdf"  # Corrected the path to your resume PDF

    try:
        resume_text = extract_text_from_pdf(resume_path)
        similarity = calculate_similarity(job_description, resume_text)
        print(f"Similarity score: {similarity}")
    except FileNotFoundError:
        print(f"Error: File '{resume_path}' not found.")

Similarity score: 0.9012153744697571
